# Data Analytics Tool and Pre-requisites

In [1]:
import numpy as np
import tqdm
import re

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('consumer_complaints.csv')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
unique_labels = sorted(list( df['product'].unique()))

In [5]:
unique_labels

['Bank account or service',
 'Consumer Loan',
 'Credit card',
 'Credit reporting',
 'Debt collection',
 'Money transfers',
 'Mortgage',
 'Other financial service',
 'Payday loan',
 'Prepaid card',
 'Student loan']

In [6]:
def label_maker(x):
    for i,c in enumerate(unique_labels):
        if c == x:
            return i

# Cleaning up the data

In [7]:
df = df.fillna('na')

In [ ]:
x_data = []
y_data = []
for i in tqdm.tqdm(df['complaint_id']):
    if df[df['complaint_id'] == i]['consumer_complaint_narrative'].item() == 'na':
        continue
    else:
        x_data.append(df[df['complaint_id'] == i]['consumer_complaint_narrative'].item())
        y_data.append(df[df['complaint_id'] == i]['product'].item())
del df

  0%|          | 0/555957 [00:00<?, ?it/s]/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.
 34%|███▍      | 190116/555957 [03:50<05:47, 1053.38it/s]/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys
/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  
 62%|██████▏   | 342844/555957 [08:47<03:48, 934.68it/s] 

In [ ]:
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
stopword = set(stopwords.words('english'))
from nltk import word_tokenize

In [ ]:
for i,issue in tqdm.tqdm(enumerate(x_data)):
    x_data[i] = x_data[i].lower()
    x_data[i] = re.sub(r'[^\w]', ' ', x_data[i])
    x_data[i] = word_tokenize(x_data[i])
    x_data[i] = [w for w in x_data[i] if w not in stopword]

In [ ]:
from itertools import chain

In [ ]:
model = Word2Vec(x_data, size=50, window=5, iter=5)

In [ ]:
for i, issue in tqdm.tqdm(enumerate(x_data)):
    n = len(issue)
    for j in range(50-n):
        x_data[i].append(' ')
    if n>50:
        x_data[i] = x_data[i][:50]

In [ ]:
len(x_data[1280])

In [ ]:
print(model)

In [ ]:
for i,issue in tqdm.tqdm(enumerate(x_data)):
    for j,word in enumerate(x_data[i]):
        if word in model.wv.vocab.keys():
            x_data[i][j] = model[word]
        else:
            x_data[i][j] = [0]*50

In [ ]:
model.most_similar(positive= ['card'])

In [ ]:
model.save('w2v_embedding.model')

In [ ]:
out = []
ct = 0
file = 0
n = len(x_data)
for i in tqdm.tqdm(range(len(x_data))):
    out.append((x_data[i],label_maker(y_data[i])))
    ct+=1
    n-=1
    if ct == 10000 or n == 0:
        np.save('npy/'+str(file) + '.npy', out, allow_pickle = True)
        file+=1
        ct = 0
        del out[:]     

In [ ]:
del x_data, y_data

In [ ]:
model = Word2Vec.load('w2v_embedding.model')

In [ ]:
from itertools import chain

In [ ]:
from sklearn.decomposition import SparsePCA, PCA, TruncatedSVD, KernelPCA, IncrementalPCA

In [ ]:
transformer = IncrementalPCA(n_components=50, batch_size=100)

In [ ]:
for i in range(7):
    x = []
    for d,l in tqdm.tqdm(np.load('npy/'+str(i)+'.npy', allow_pickle=True)):
        x.append(list(chain.from_iterable(d)))
    transformer.partial_fit(x)

In [ ]:
from joblib import dump, load

In [ ]:
dump(transformer, 'PCA.joblib')

In [ ]:
transformer_ = load('PCA.joblib')
print(transformer_)

In [ ]:
for i in tqdm.tqdm(range(7)):    
    out = []
    data = []
    labels = []
    x = np.load('npy/'+str(i)+'.npy', allow_pickle=True)
    for d in x:
        data.append(list(chain.from_iterable(d[0])))
        labels.append(d[1])
    data = transformer_.transform(data)
    for j in range(len(x)):
        out.append((data[j],labels[j]))
    np.save('npy-features/'+str(i) + '.npy', out, allow_pickle = True)